In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat


In [2]:
from pyod.models.abod import ABOD

In [3]:
pip install pyod

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.mcd import MCD
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [5]:
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']


In [6]:
data=loadmat('E:\data\cardio.mat')

In [7]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [8]:
len(data)

5

In [9]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [11]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [14]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

In [17]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [18]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']

In [19]:
roc_df=pd.DataFrame(columns=df_columns)

In [20]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [21]:
prn_df=pd.DataFrame(columns=df_columns)

In [22]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [23]:
time_df=pd.DataFrame(columns=df_columns)

In [24]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [38]:
from time import time
random_state=np.random.RandomState(42)

for mat_file in mat_file_list:
    print('....processing',mat_file,'....')
    mat = loadmat(os.path.join('E:\data',mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y)/len(y)
    outliers_percentage = round(outlier_fraction *100 ,ndigits=4)
    
     # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

     # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)
    X_train_norm , X_test_norm = standardizer(X_train,X_test)
    
    classifiers = {'Angle Based Outlier Detector (ABOD)': ABOD(contamination=outlier_fraction)
                  ,'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
                  'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
                   'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
                  }
    
    for clf_name, clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)
        
        
        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn},'
             'execution_time:{duration}s'.format(
             clf_name=clf,roc=roc,prn=prn,duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df_columns=df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)

....processing arrhythmia.mat ....
ABOD(contamination=0.14601769911504425, method='fast', n_neighbors=5) ROC:0.7687, precision @ rank n:0.3571,execution_time:0.1285s
CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.14601769911504425, n_clusters=8, n_jobs=1,
   random_state=RandomState(MT19937) at 0x1EDD8F2EE40, use_weights=False) ROC:0.7684, precision @ rank n:0.4643,execution_time:0.0967s
FeatureBagging(base_estimator=None, bootstrap_features=False,
        check_detector=True, check_estimator=False, combination='average',
        contamination=0.14601769911504425, estimator_params={},
        max_features=1.0, n_estimators=10, n_jobs=1,
        random_state=RandomState(MT19937) at 0x1EDD8F2EE40, verbose=0) ROC:0.7799, precision @ rank n:0.5,execution_time:0.4883s
HBOS(alpha=0.1, contamination=0.14601769911504425, n_bins=10, tol=0.5) ROC:0.8511, precision @ rank n:0.5714,execution_time:0.062s
IForest(behaviour='old', bootstrap=False, contam

IForest(behaviour='old', bootstrap=False, contamination=0.358974358974359,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=1,
    random_state=RandomState(MT19937) at 0x1EDD8F2EE40, verbose=0) ROC:0.8294, precision @ rank n:0.6607,execution_time:0.3006s
KNN(algorithm='auto', contamination=0.358974358974359, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=5, p=2, radius=1.0) ROC:0.9134, precision @ rank n:0.8393,execution_time:0.0312s
LOF(algorithm='auto', contamination=0.358974358974359, leaf_size=30,
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=20, p=2) ROC:0.8989, precision @ rank n:0.75,execution_time:0.0s
MCD(assume_centered=False, contamination=0.358974358974359,
  random_state=RandomState(MT19937) at 0x1EDD8F2EE40, store_precision=True,
  support_fraction=None) ROC:0.9399, precision @ rank n:0.8571,execution_time:0.0594s
OCSVM(cache_size=200, coef0=0.0, contamination=0.358974358974359, degree=

ABOD(contamination=0.14601769911504425, method='fast', n_neighbors=5) ROC:0.0809, precision @ rank n:0.0333,execution_time:1.8065s
CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.03167864141084259, n_clusters=8, n_jobs=1,
   random_state=RandomState(MT19937) at 0x1EDD8F2EE40, use_weights=False) ROC:1.0, precision @ rank n:1.0,execution_time:0.1739s
FeatureBagging(base_estimator=None, bootstrap_features=False,
        check_detector=True, check_estimator=False, combination='average',
        contamination=0.03167864141084259, estimator_params={},
        max_features=1.0, n_estimators=10, n_jobs=1,
        random_state=RandomState(MT19937) at 0x1EDD8F2EE40, verbose=0) ROC:0.5228, precision @ rank n:0.1667,execution_time:10.318s
HBOS(alpha=0.1, contamination=0.03167864141084259, n_bins=10, tol=0.5) ROC:0.9999, precision @ rank n:0.9667,execution_time:0.0469s
IForest(behaviour='old', bootstrap=False, contamination=0.03167864141084259,
    max_

IForest(behaviour='old', bootstrap=False, contamination=0.3489583333333333,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=1,
    random_state=RandomState(MT19937) at 0x1EDD8F2EE40, verbose=0) ROC:0.6777, precision @ rank n:0.4787,execution_time:0.3115s
KNN(algorithm='auto', contamination=0.3489583333333333, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=5, p=2, radius=1.0) ROC:0.7252, precision @ rank n:0.5106,execution_time:0.0312s
LOF(algorithm='auto', contamination=0.3489583333333333, leaf_size=30,
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=20, p=2) ROC:0.6604, precision @ rank n:0.4787,execution_time:0.0s
MCD(assume_centered=False, contamination=0.3489583333333333,
  random_state=RandomState(MT19937) at 0x1EDD8F2EE40, store_precision=True,
  support_fraction=None) ROC:0.7047, precision @ rank n:0.4787,execution_time:0.0469s
OCSVM(cache_size=200, coef0=0.0, contamination=0.3489583333333333, 

OCSVM(cache_size=200, coef0=0.0, contamination=0.0715114976475141, degree=3,
   gamma='auto', kernel='rbf', max_iter=-1, nu=0.5, shrinking=True,
   tol=0.001, verbose=False) ROC:0.9922, precision @ rank n:0.9553,execution_time:43.1018s
PCA(contamination=0.0715114976475141, copy=True, iterated_power='auto',
  n_components=None, n_selected_components=None,
  random_state=RandomState(MT19937) at 0x1EDD8F2EE40, standardization=True,
  svd_solver='auto', tol=0.0, weighted=True, whiten=False) ROC:0.9902, precision @ rank n:0.9503,execution_time:0.0312s
....processing vertebral.mat ....
ABOD(contamination=0.14601769911504425, method='fast', n_neighbors=5) ROC:0.2797, precision @ rank n:0.0,execution_time:0.0469s
CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.125, n_clusters=8, n_jobs=1,
   random_state=RandomState(MT19937) at 0x1EDD8F2EE40, use_weights=False) ROC:0.3908, precision @ rank n:0.0,execution_time:0.0469s
FeatureBagging(base_estimator=

In [40]:
roc_df


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,14.6018,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,14.6018,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,14.6018,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,14.6018,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,14.6018,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,14.6018,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,14.6018,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,14.6018,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971


In [41]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,14.6018,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,14.6018,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,14.6018,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,14.6018,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,14.6018,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,14.6018,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,14.6018,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,14.6018,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893


In [42]:
time_df

,0,1,2,3,4,5,6,7,8,9,...,Data,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,Outlier Perc,PCA
0,arrhythmia,452,274,14.6018,2.7367,1.9919,0.5119,1.553,0.4684,0.0781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,cardio,1831,21,14.6018,0.353,0.1237,0.7243,0.008,0.423,0.1543,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,glass,214,9,14.6018,0.0475,0.039,0.0312,0,0.3244,0.009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ionosphere,351,33,14.6018,0.066,0.0524,0.0734,0.009,0.3309,0.018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,letter,1600,32,14.6018,0.3388,0.1115,0.6906,0.01,0.4288,0.1279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,lympho,148,18,14.6018,0.025,0.04,0.031,0.005,0.2898,0.007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,arrhythmia,452,274,14.6018,0.1285,0.0967,0.4883,0.062,0.3781,0.0865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,cardio,1831,21,14.6018,0.3586,0.1094,0.7265,0,0.4288,0.1381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,glass,214,9,14.6018,0.0312,0.0396,0.0358,0,0.2968,0.009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ionosphere,351,33,14.6018,0.062,0.047,0.069,0.008,0.3006,0.0312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['wbc',
 378,
 30,
 14.6018,
 0.9232,
 0.9063,
 0.9415,
 0.9592,
 0.9451,
 0.9437,
 0.9352,
 0.8986,
 0.9408,
 0.9324]

In [45]:
classifiers.items()

dict_items([('Angle Based Outlier Detector (ABOD)', ABOD(contamination=0.14601769911504425, method='fast', n_neighbors=5)), ('Cluster-based Local Outlier Factor', CBLOF(alpha=0.9, beta=5, check_estimator=False, clustering_estimator=None,
   contamination=0.05555555555555555, n_clusters=8, n_jobs=1,
   random_state=RandomState(MT19937) at 0x1EDD8F2EE40, use_weights=False)), ('Feature Bagging', FeatureBagging(base_estimator=None, bootstrap_features=False,
        check_detector=True, check_estimator=False, combination='average',
        contamination=0.05555555555555555, estimator_params={},
        max_features=1.0, n_estimators=10, n_jobs=1,
        random_state=RandomState(MT19937) at 0x1EDD8F2EE40, verbose=0)), ('Histogram-base Outlier Detection (HBOS)', HBOS(alpha=0.1, contamination=0.05555555555555555, n_bins=10, tol=0.5)), ('Isolation Forest', IForest(behaviour='old', bootstrap=False, contamination=0.05555555555555555,
    max_features=1.0, max_samples='auto', n_estimators=100, n_